In [1]:
import itertools
from math import sqrt, gcd, lcm

# 91 - Right Triangles with Integer Coordinates

In [2]:
%%time
def pe_91():
    limit = 50
    points = []
    for i in range(limit+1):
        for j in range(limit+1):
            if i == 0 and j == 0: continue
            points.append((i,j))
    total = 0
    for i, (a, b) in enumerate(points):
        for c, d in points[i+1:]:
            p, q, r = sorted([a**2 + b**2, c**2 + d**2, ((a-c)**2) + ((b-d)**2)])
            if p + q - r == 0: total += 1
    print(total)
pe_91()

14234
CPU times: user 1 s, sys: 0 ns, total: 1 s
Wall time: 1 s


# 92 - Square Digit Chains

In [3]:
%%time
def pe_92():
    nums_89 = []
    squares = [i*i for i in range(10)]
    for s in range(2, 568):
        num = s
        while True:
            if num == 89:
                nums_89.append(s)
                break
            elif num == 1:
                break
            else:
                num = sum([squares[int(c)] for c in str(num)])
    nums_89 = set(nums_89) 
    total = 0
    for s in range(2, 10**7):
        if sum([squares[int(c)] for c in str(s)]) in nums_89: total += 1
    print(total)
pe_92()

8581146
CPU times: user 5.87 s, sys: 1.28 ms, total: 5.88 s
Wall time: 5.87 s


# 93 - Arithmetic Expressions

In [4]:
def eval_n(digits, ops):
    result = []
    t_d = {c:k for c,k in zip(['a', 'b', 'c', 'd', 'x', 'y', 'z'], digits + ops)}    
    templates = [
        "( {a} {x} ( {b} {y} {c} )) {z} {d}",
        "( {a} {x} {b} ) {y} ( {c} {z} {d} )",
        "(( {a} {x} {b} ) {y} {c} ) {z} {d}",
        "{a} {x} ( {b} {y} ( {c} {z} {d} ))",
        "{a} {x} (( {b} {y} {c} ) {z} {d} )"  
    ]
    for template in templates:
        try:
            result.append(eval(template.format(**t_d)))
        except ZeroDivisionError:
            continue
    return result

In [5]:
%%time
def pe_93():
    max_n = ("", 0)
    ops = ["*", "/", "+", "-"]
    for digits in itertools.combinations(range(1,10), 4):
        nums_made = set()
        for operators in itertools.product(range(0,4), repeat=3):
            for o_perm in itertools.permutations(operators):
                o = [ops[i] for i in list(o_perm)]
                for d_perm in itertools.permutations(digits):
                    for num in eval_n(list(d_perm), o): nums_made.add(num)
        n = 1
        while n in nums_made: n+=1
        n-=1
        if n > max_n[1]: max_n = (str(''.join(str(k) for k in digits)), n)
    print(max_n)
pe_93()

('1258', 51)
CPU times: user 42.7 s, sys: 184 ms, total: 42.9 s
Wall time: 42.9 s


# 94 - Almost Equilateral Triangles

In [6]:
%%time
def pe_94():
    total = 0
    n_limit = 10**9
    limit = int(sqrt((10**9)/6)) + 100
    for n in range(1,limit):
        for m in range(n+1,limit):
            if not (n%2 ^ m%2) or gcd(n,m)!=1: continue
            a,b,c = [m**2 - n**2, 2*m*n, m**2 + n**2]
            if 2*a == c-1 or 2*a == c+1:
                sum_n = 2*a + 2*c
                if sum_n > n_limit: break
                total += sum_n
            elif 2*b == c-1 or 2*b == c+1:
                sum_n = 2*b + 2*c
                if sum_n > n_limit: break
                total += sum_n
    print(total)
pe_94()

518408346
CPU times: user 14.5 s, sys: 32 ms, total: 14.5 s
Wall time: 14.5 s


# 95 - Amicable Chains

In [7]:
%%time
def pe_95():
    max_c = (0,0)
    limit = 10**6
    sum_d = {i:0 for i in range(1,limit+1)}
    for i in range(1, limit//2 + 1):
        k=2
        while k*i <= limit:
            sum_d[k*i] += i
            k+=1
    chain_d = {i:0 for i in range(1,limit+1)}
    for i in range(2, limit+1):
        if not chain_d[i]:
            this_chain = [i]
            curr_num = i
            while True:
                prev = curr_num
                curr_num = sum_d[curr_num]
                if curr_num == prev or curr_num == 1: break
                if curr_num > limit:
                    for num in this_chain: chain_d[num] = 1
                    break
                elif curr_num in this_chain:
                    for num in this_chain: chain_d[num] = 1
                    this_chain = this_chain[this_chain.index(curr_num):]
                    chain_len = len(this_chain)
                    if chain_len >= max_c[0]: max_c = (chain_len, min(this_chain))
                    break
                this_chain.append(curr_num)
    print(max_c[1])
pe_95()

14316
CPU times: user 4.82 s, sys: 92 ms, total: 4.91 s
Wall time: 4.91 s


# 96 - Su Doku

In [8]:
def reduce(grid):
    for _ in range(10):
        solved = True
        for row in grid:
            for item in row:
                if len(item) != 1:
                    solved = False
                    break
            if not solved: break
        if solved: return 1
        for i,row in enumerate(grid):
            for j,item in enumerate(row):
                if len(item) == 1:
                    num = item[0]
                    for k in range(9):
                        if num in grid[k][j] and k!=i: grid[k][j].remove(num)
                    for k in range(9):
                        if num in grid[i][k] and k!=j: grid[i][k].remove(num)
                    for x in range(3*(i//3), 3*(i//3) + 3):
                        for y in range(3*(j//3), 3*(j//3) + 3):
                            if num in grid[x][y] and (x,y)!=(i,j): grid[x][y].remove(num)
        for col in range(9):
            for digit in range(1,10):
                found_in_rows = []
                for row in range(9):
                    if digit in grid[row][col]: found_in_rows.append(row)
                if len(found_in_rows) == 1:
                    row = found_in_rows[0]
                    grid[row][col] = [digit]
        for row in range(9):
            for digit in range(1,10):
                found_in_cols = []
                for col in range(9):
                    if digit in grid[row][col]: found_in_cols.append(col)
                if len(found_in_cols) == 1:
                    col = found_in_cols[0]
                    grid[row][col] = [digit]
        for a in range(3):
            for b in range(3):
                for digit in range(1,10):
                    found_in_locs = []
                    for x in range(3):
                        row = 3*a + x
                        for y in range(3):
                            col = 3*b + y
                            if digit in grid[row][col]: found_in_locs.append((row, col))
                    if len(found_in_locs)==1:
                        row, col = found_in_locs[0]
                        grid[row][col] = [digit]
    return 0

In [9]:
def is_valid(grid, row, col, digit):
    for i in range(9):
        if i == row: continue
        item = grid[i][col]
        if len(item) == 1 and item[0] == digit: return False
    for j in range(9):
        if j == col: continue
        item = grid[row][j]
        if len(item) == 1 and item[0] == digit: return False
    startRow = row - (row % 3)
    startCol = col - (col % 3);
    for i in range(3):
        for j in range(3):
            item = grid[i + startRow][j + startCol]
            if item[0] == digit and len(item) == 1: return False
    return True

In [10]:
def solve(grid, row, col):
    if row == 8 and col == 9: return True
    if col == 9:
        row = row + 1
        col = 0
    if len(grid[row][col]) == 1: return solve(grid, row, col+1)
    list_t = grid[row][col]
    og_list = list_t[:]
    for digit in grid[row][col]:
        if is_valid(grid, row, col, digit):
            grid[row][col] = [digit]
            if solve(grid, row, col+1): return True
    grid[row][col] = og_list[:]
    return False

In [11]:
%%time
def pe_96():
    grids = []
    this_grid = []
    with open("./inputs/0096_sudoku.txt", 'r') as file:
        for line in file:
            if line.startswith("Grid"):
                if this_grid: grids.append(this_grid)
                this_grid = []
            else:
                row = [[int(c)] for c in line.strip()]
                for w in row:
                    if w[0] == 0: w[:]=[1,2,3,4,5,6,7,8,9]
                this_grid.append(row)
        grids.append(this_grid)
        
    total = 0
    for i,grid in enumerate(grids): 
        if not reduce(grid): solve(grid, 0, 0)
        total += int(''.join([str(c[0]) for c in grid[0][:3]]))
    print(total)
pe_96()

24702
CPU times: user 146 ms, sys: 14 μs, total: 146 ms
Wall time: 145 ms


# 97 - Large Non-Mersenne Prime

In [12]:
%%time
def pe_97():
    print(((28433*(2**7830457))+1)%(10**10))
pe_97()

8739992577
CPU times: user 19.1 ms, sys: 0 ns, total: 19.1 ms
Wall time: 19 ms


# 98 - Anagramic Squares

In [13]:
class Word:
    def __init__(self, word):
        self.word = word
        self.word_len = len(word)
        self.word_arr = [c for c in word]
        self.word_chars = tuple(sorted(self.word_arr))
    def __eq__(self, other):
        return self.word_chars == other.word_chars and not tuple(self.word_arr) == tuple(reversed(other.word_arr))

In [14]:
def get_max(word1, word2, n_squares):
    max_s = 0
    for n in n_squares:
        char_map = {k:v for k,v in zip(word1.word_arr, [c for c in str(n)])}
        if len(set(list(char_map.values()))) != len(list(char_map.values())): continue
        new_num = ''.join([char_map[c] for c in word2.word_arr])
        if new_num[0] == '0': continue
        new_num = int(new_num)
        if new_num in n_squares:
            t_max = max(n, new_num)
            if t_max > max_s: max_s = t_max
    return max_s

In [15]:
%%time
def pe_98():
    with open("./inputs/0098_words.txt", 'r') as file:
        contents = [Word(c[1:-1]) for c in file.read().split(',')]
    square_lists = {i:[] for i in range(11)}
    for i in range(10**5): square_lists[len(str(i*i))].append(i*i)
    max_n = 0
    for i,word1 in enumerate(contents):
        for word2 in contents[i+1:]:
            if word1 == word2:
                n = get_max(word1, word2, square_lists[word1.word_len])
                if n > max_n: max_n = n
    print(max_n)
pe_98()

18769
CPU times: user 183 ms, sys: 0 ns, total: 183 ms
Wall time: 182 ms


# 99 - Largest Exponential

In [16]:
%%time
def pe_99():
    max_n = (0, 0, 0)
    with open("./inputs/0099_base_exp.txt", 'r') as file:
        for i,line in enumerate(file):
            base, exp = [int(c) for c in line.strip().split(',')]
            if base > max_n[0] and exp > max_n[1]:
                max_n = (base, exp, i)
                continue
            div = min(max_n[1], exp)
            a = base ** (exp/div)
            b = max_n[0] ** (max_n[1]/div)
            if a > b: max_n = (base, exp, i)
    print(max_n[2] + 1)
pe_99()

709
CPU times: user 1.3 ms, sys: 0 ns, total: 1.3 ms
Wall time: 1.25 ms


# 100 - Arranged Probability

In [17]:
%%time
def pe_100():
    b = 0
    t = 0
    b_0 = 15
    t_0 = 21
    while t < 10**12:
        b = (3*b_0) + (2*t_0) - 2
        t = (4*b_0) + (3*t_0) - 3
        b_0 = b
        t_0 = t
    print(b)
pe_100()

756872327473
CPU times: user 108 μs, sys: 0 ns, total: 108 μs
Wall time: 117 μs
